In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from datetime import datetime

In [2]:
idf_df = pd.read_csv('transac_idf_cleaned.csv')
# db cleaned is without all real estate > 30000€/m2

In [3]:
# define potentials data X => run test on all
X = idf_df[['longitude', 'latitude','n_pieces','type_bat']].values
y = idf_df['prix_m2'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X

array([[ 2.40375064, 48.83624849,  2.        ,  1.        ],
       [ 2.348148  , 48.86560434,  1.        ,  1.        ],
       [ 2.32468916, 48.88330859,  3.        ,  1.        ],
       ...,
       [ 2.21748065, 48.99549225,  2.        ,  1.        ],
       [ 2.3147687 , 48.97772502,  4.        ,  1.        ],
       [ 2.21243488, 49.0221078 ,  3.        ,  1.        ]])

In [6]:
y

array([10997.        , 11630.76923077, 11271.18644068, ...,
        2443.39622642,  6546.39175258,  2428.57142857])

In [7]:
from sklearn.model_selection import GridSearchCV

params_grid = {
                'LR': {
                    'model': LinearRegression(),
                    'params': {
                        'fit_intercept': [True, False],
                        'positive': [True, False]
                    }
                },
                'DTR': {
                    'model': DecisionTreeRegressor(),
                    'params': {
                        'max_depth': list(range(4, 8)),
                        'min_samples_leaf': list(range(25,50))
                    }
                },
                'RFR': {
                    'model': RandomForestRegressor(),
                    'params': {
                        'max_depth': [25, 50]
                    }
                } 
            }

for model_name, model_config in params_grid.items():
    gs = GridSearchCV(estimator=model_config['model'], 
                      param_grid=model_config['params'])
    gs.fit(X_train, y_train)
    print(f'Modèle: {model_name} avec params optimaux: {gs.best_params_} donne erreur =')
    print(np.sqrt(mean_squared_error(y_test, gs.best_estimator_.predict(X_test))))

Modèle: LR avec params optimaux: {'fit_intercept': True, 'positive': False} donne erreur =
3762.083911769643
Modèle: DTR avec params optimaux: {'max_depth': 7, 'min_samples_leaf': 25} donne erreur =
2316.2477799342596
Modèle: RFR avec params optimaux: {'max_depth': 25} donne erreur =
2088.1639427678324


In [8]:
best_model_fit = DecisionTreeRegressor(max_depth=7, min_samples_leaf=49)

In [9]:
pd.DataFrame(best_model_fit.feature_importances_, index=['longitude', 'latitude', 'n_pieces', 'type_bat'], columns=['Importance']).plot.bar()

NotFittedError: This DecisionTreeRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.